In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Librerías necesarias
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
import joblib

# Cargar el dataset
dataset_path = "/content/drive/MyDrive/TFM_Kschool/DataSets/DAIGT V2/train_v2_drcat_02.csv"
data = pd.read_csv(dataset_path)

# Mostrar información del dataset
print("Dataset cargado correctamente.")
print(f"Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
print(data.head())

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Dataset cargado correctamente.
Filas: 44868, Columnas: 5
                                                text  label  \
0  Phones\n\nModern humans today are always on th...      0   
1  This essay will explain if drivers should or s...      0   
2  Driving while the use of cellular devices\n\nT...      0   
3  Phones & Driving\n\nDrivers should not be able...      0   
4  Cell Phone Operation While Driving\n\nThe abil...      0   

          prompt_name           source  RDizzl3_seven  
0  Phones and driving  persuade_corpus          False  
1  Phones and driving  persuade_corpus          False  
2  Phones and driving  persuade_corpus          False  
3  Phones and driving  persuade_corpus          False  
4  Phones and driving  persuade_corpus          False  


In [3]:
# Ensure 'text' column is treated as strings, even if it contains NaNs
data['text'] = data['text'].astype(str).fillna("").str.strip()

# Mostrar estadísticas
print("\nEstadísticas del dataset de entrenamiento:")
print(f"Total muestras: {len(data)}")
print(f"Textos AI: {sum(data['label'] == 1)}")
print(f"Textos humanos: {sum(data['label'] == 0)}")
# Make sure 'train' is actually 'data'
print(f"Ratio AI/Humano: {sum(data['label'] == 1)/sum(data['label'] == 0):.3f}")


Estadísticas del dataset de entrenamiento:
Total muestras: 44868
Textos AI: 17497
Textos humanos: 27371
Ratio AI/Humano: 0.639


In [4]:
import re

# Función para pretratamiento con eliminación de signos de puntuación y caracteres especiales
def clean_text_remove_specials(text):
    # Eliminar caracteres que no sean letras o números
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Mantiene solo letras, números y espacios
    text = re.sub(r"\s+", " ", text).strip()  # Elimina espacios adicionales
    return text.lower()

# Función para pretratamiento sin eliminar signos de puntuación
def clean_text_keep_specials(text):
    # Mantener el texto, solo transformaciones mínimas
    text = re.sub(r"\s+", " ", text).strip()  # Elimina espacios adicionales
    return text.lower()

# Crear dos nuevas columnas con los textos preprocesados
data['text_clean_specials_removed'] = data['text'].apply(clean_text_remove_specials)
data['text_clean_keep_specials'] = data['text'].apply(clean_text_keep_specials)

# Verificar las primeras filas
print(data.head())

                                                text  label  \
0  Phones\n\nModern humans today are always on th...      0   
1  This essay will explain if drivers should or s...      0   
2  Driving while the use of cellular devices\n\nT...      0   
3  Phones & Driving\n\nDrivers should not be able...      0   
4  Cell Phone Operation While Driving\n\nThe abil...      0   

          prompt_name           source  RDizzl3_seven  \
0  Phones and driving  persuade_corpus          False   
1  Phones and driving  persuade_corpus          False   
2  Phones and driving  persuade_corpus          False   
3  Phones and driving  persuade_corpus          False   
4  Phones and driving  persuade_corpus          False   

                         text_clean_specials_removed  \
0  phones modern humans today are always on their...   
1  this essay will explain if drivers should or s...   
2  driving while the use of cellular devices toda...   
3  phones driving drivers should not be able to u...  

In [5]:

# Cargar el tokenizer de DeBERTa
model_name = "microsoft/deberta-v3-base"  # Puedes elegir otra variante de DeBERTa
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizar los textos preprocesados
# Aquí usaremos las dos columnas pretratadas
data['tokens_specials_removed'] = data['text_clean_specials_removed'].apply(
    lambda x: tokenizer.tokenize(x)
)
data['tokens_keep_specials'] = data['text_clean_keep_specials'].apply(
    lambda x: tokenizer.tokenize(x)
)

# Mostrar las primeras filas con los tokens generados
print(data[['text', 'tokens_specials_removed', 'tokens_keep_specials']].head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


                                                text  \
0  Phones\n\nModern humans today are always on th...   
1  This essay will explain if drivers should or s...   
2  Driving while the use of cellular devices\n\nT...   
3  Phones & Driving\n\nDrivers should not be able...   
4  Cell Phone Operation While Driving\n\nThe abil...   

                             tokens_specials_removed  \
0  [▁phones, ▁modern, ▁humans, ▁today, ▁are, ▁alw...   
1  [▁this, ▁essay, ▁will, ▁explain, ▁if, ▁drivers...   
2  [▁driving, ▁while, ▁the, ▁use, ▁of, ▁cellular,...   
3  [▁phones, ▁driving, ▁drivers, ▁should, ▁not, ▁...   
4  [▁cell, ▁phone, ▁operation, ▁while, ▁driving, ...   

                                tokens_keep_specials  
0  [▁phones, ▁modern, ▁humans, ▁today, ▁are, ▁alw...  
1  [▁this, ▁essay, ▁will, ▁explain, ▁if, ▁drivers...  
2  [▁driving, ▁while, ▁the, ▁use, ▁of, ▁cellular,...  
3  [▁phones, ▁&, ▁driving, ▁drivers, ▁should, ▁no...  
4  [▁cell, ▁phone, ▁operation, ▁while, ▁driving, ..

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset

# Configuración
MODEL_NAME = "microsoft/deberta-v3-base"  # Modelo DeBERTa
BATCH_SIZE = 32  # Tamaño del batch
MAX_LENGTH = 512  # Longitud máxima de tokens
CHUNK_SIZE = 1000  # Tamaño de cada fragmento del dataset

# Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# Mover modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Función para tokenizar textos para una columna específica
def tokenize_column(data, column_name, max_length):
    """
    Tokeniza los textos en una columna específica del dataset.
    Args:
        data (DataFrame): Dataset que contiene los textos.
        column_name (str): Nombre de la columna con los textos.
        max_length (int): Longitud máxima de los tokens.
    Returns:
        dict: Tensores tokenizados.
    """
    return tokenizer(
        data[column_name].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Función para dividir el dataset en fragmentos
def chunk_data(data, chunk_size):
    """
    Divide el dataset en fragmentos más pequeños.
    Args:
        data (DataFrame): El dataset completo.
        chunk_size (int): Tamaño de cada fragmento.
    Returns:
        List[DataFrame]: Lista de fragmentos.
    """
    return [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# Crear DataLoader para batches
def create_dataloader(tokens, batch_size):
    dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'])
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Generar embeddings
def generate_embeddings(dataloader, model, device):
    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = [t.to(device, non_blocking=True) for t in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_size)
            embeddings.append(cls_embeddings)
            del input_ids, attention_mask, outputs
            torch.cuda.empty_cache()
    return torch.cat(embeddings, dim=0)

# Fragmentar el dataset
data_chunks = chunk_data(data, CHUNK_SIZE)

# Procesar cada fragmento por separado
for i, chunk in enumerate(data_chunks):
    print(f"Procesando fragmento {i + 1}/{len(data_chunks)}...")

    # Tokenizar el fragmento
    tokens_chunk = tokenize_column(chunk, 'text_clean_specials_removed', MAX_LENGTH)
    dataloader_chunk = create_dataloader(tokens_chunk, BATCH_SIZE)

    # Generar embeddings para el fragmento
    embeddings_chunk = generate_embeddings(dataloader_chunk, model, device)

    # Guardar los embeddings del fragmento
    torch.save(
        embeddings_chunk,
        f"/content/drive/MyDrive/TFM_Kschool/DataSets/Dmitiva/data6k/DeBerta_train/embeddings_specials_removed_validation_chunk_{i + 1}.pt"
    )

    # Liberar memoria
    del tokens_chunk, dataloader_chunk, embeddings_chunk
    torch.cuda.empty_cache()

print("Todos los fragmentos han sido procesados y guardados correctamente.")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Procesando fragmento 1/45...
Procesando fragmento 2/45...
Procesando fragmento 3/45...
Procesando fragmento 4/45...
Procesando fragmento 5/45...
Procesando fragmento 6/45...
Procesando fragmento 7/45...
Procesando fragmento 8/45...
Procesando fragmento 9/45...
Procesando fragmento 10/45...
Procesando fragmento 11/45...
Procesando fragmento 12/45...
Procesando fragmento 13/45...
Procesando fragmento 14/45...
Procesando fragmento 15/45...
Procesando fragmento 16/45...
Procesando fragmento 17/45...
Procesando fragmento 18/45...
Procesando fragmento 19/45...
Procesando fragmento 20/45...
Procesando fragmento 21/45...
Procesando fragmento 22/45...
Procesando fragmento 23/45...
Procesando fragmento 24/45...
Procesando fragmento 25/45...
Procesando fragmento 26/45...
Procesando fragmento 27/45...
Procesando fragmento 28/45...
Procesando fragmento 29/45...
Procesando fragmento 30/45...
Procesando fragmento 31/45...
Procesando fragmento 32/45...
Procesando fragmento 33/45...
Procesando fragment

In [7]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset

# Configuración
MODEL_NAME = "microsoft/deberta-v3-base"  # Modelo DeBERTa
BATCH_SIZE = 16  # Tamaño del batch
MAX_LENGTH = 512  # Longitud máxima de tokens
CHUNK_SIZE = 1000  # Tamaño de cada fragmento del dataset

# Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# Mover modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Función para tokenizar textos para una columna específica
def tokenize_column(data, column_name, max_length):
    """
    Tokeniza los textos en una columna específica del dataset.
    Args:
        data (DataFrame): Dataset que contiene los textos.
        column_name (str): Nombre de la columna con los textos.
        max_length (int): Longitud máxima de los tokens.
    Returns:
        dict: Tensores tokenizados.
    """
    return tokenizer(
        data[column_name].tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Función para dividir el dataset en fragmentos
def chunk_data(data, chunk_size):
    """
    Divide el dataset en fragmentos más pequeños.
    Args:
        data (DataFrame): El dataset completo.
        chunk_size (int): Tamaño de cada fragmento.
    Returns:
        List[DataFrame]: Lista de fragmentos.
    """
    return [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# Crear DataLoader para batches
def create_dataloader(tokens, batch_size):
    dataset = TensorDataset(tokens['input_ids'], tokens['attention_mask'])
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Generar embeddings
def generate_embeddings(dataloader, model, device):
    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask = [t.to(device, non_blocking=True) for t in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_size)
            embeddings.append(cls_embeddings)
            del input_ids, attention_mask, outputs
            torch.cuda.empty_cache()
    return torch.cat(embeddings, dim=0)

# Fragmentar el dataset
data_chunks = chunk_data(data, CHUNK_SIZE)

# Procesar cada fragmento por separado
for i, chunk in enumerate(data_chunks):
    print(f"Procesando fragmento {i + 1}/{len(data_chunks)}...")

    # Tokenizar el fragmento
    tokens_chunk = tokenize_column(chunk, 'text_clean_keep_specials', MAX_LENGTH)
    dataloader_chunk = create_dataloader(tokens_chunk, BATCH_SIZE)

    # Generar embeddings para el fragmento
    embeddings_chunk = generate_embeddings(dataloader_chunk, model, device)

    # Guardar los embeddings del fragmento
    torch.save(
        embeddings_chunk,
        f"/content/drive/MyDrive/TFM_Kschool/DataSets/Dmitiva/data6k/DeBerta_train/embeddings_keep_specials_validation_chunk_{i + 1}.pt"
    )

    # Liberar memoria
    del tokens_chunk, dataloader_chunk, embeddings_chunk
    torch.cuda.empty_cache()

print("Todos los fragmentos para 'text_clean_keep_specials' han sido procesados y guardados correctamente.")


Procesando fragmento 1/45...
Procesando fragmento 2/45...
Procesando fragmento 3/45...
Procesando fragmento 4/45...
Procesando fragmento 5/45...
Procesando fragmento 6/45...
Procesando fragmento 7/45...
Procesando fragmento 8/45...
Procesando fragmento 9/45...
Procesando fragmento 10/45...
Procesando fragmento 11/45...
Procesando fragmento 12/45...
Procesando fragmento 13/45...
Procesando fragmento 14/45...
Procesando fragmento 15/45...
Procesando fragmento 16/45...
Procesando fragmento 17/45...
Procesando fragmento 18/45...
Procesando fragmento 19/45...
Procesando fragmento 20/45...
Procesando fragmento 21/45...
Procesando fragmento 22/45...
Procesando fragmento 23/45...
Procesando fragmento 24/45...
Procesando fragmento 25/45...
Procesando fragmento 26/45...
Procesando fragmento 27/45...
Procesando fragmento 28/45...
Procesando fragmento 29/45...
Procesando fragmento 30/45...
Procesando fragmento 31/45...
Procesando fragmento 32/45...
Procesando fragmento 33/45...
Procesando fragment

In [8]:
# Guardar el dataset actual como CSV
dataset_path = "/content/drive/MyDrive/TFM_Kschool/DataSets/Dmitiva/data6k/DeBerta_train/processed_data_validation.csv"  # Nombre del archivo para el dataset

# Suponiendo que `data` es tu DataFrame actual
data.to_csv(dataset_path, index=False)

print(f"Dataset guardado exitosamente como {dataset_path}. ¡Listo para continuar en otro cuaderno!")

Dataset guardado exitosamente como /content/drive/MyDrive/TFM_Kschool/DataSets/Dmitiva/data6k/DeBerta_train/processed_data_validation.csv. ¡Listo para continuar en otro cuaderno!
